In [1]:
import os
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from PIL import Image
import numpy as np
from IPython.display import Image
import torch
import PIL

In [4]:
dir_path="/content/drive/MyDrive/2기 딥러닝 프로젝트/train/"
labels = ['Closed', 'Open']
IMG_SIZE = 145
x_data = []
y_data = []
for label in labels:
   path = os.path.join(dir_path, label)
   class_num = labels.index(label)
   print(class_num)
   import torchvision.transforms as transforms
   tf = transforms.ToTensor()
   for img in os.listdir(path): # closed 폴더 안에 _1.jpg, open 폴더 안에 1.jpg 
       img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR) #/content/drive/MyDrive/2기 딥러닝 프로젝트/train/Closed/_0.jpg
       resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
       resized_array = tf(resized_array) # 해주면 image의 shape는 (3,256,256) 이된다
       resized_array = resized_array.tolist()
       x_data.append(resized_array)
       y_data.append(class_num)
    

0
1


error: ignored

In [ ]:
x_data = torch.tensor(x_data)
x_data.shape

torch.Size([1451, 3, 145, 145])

In [ ]:
y_data = np.array(y_data)
y_data = torch.tensor(y_data)
y_data.shape

torch.Size([1451])

In [ ]:
from sklearn.model_selection import train_test_split
seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, random_state=seed, test_size=test_size)

In [ ]:
X_train.shape

torch.Size([1015, 3, 145, 145])

## **train**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
#train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
#test_generator = ImageDataGenerator(rescale=1/255)

#train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
#test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(1536, 512) #k는 conv3 하고 나온 이미지의 shape를 일자로 펴준거기 때문에 아직 계산을 안 해서 이미지가 다 모아지면 계산하면 된다
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.reshape(-1, 1536)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)


        return x

In [ ]:
model = Net()
model.to('cuda')

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 50

In [ ]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0

    model.train()
    input_1 = X_train.to('cuda')
    labels = y_train.to('cuda')

    for i in range(1015):
        optimizer.zero_grad()
        outputs = model(input_1)

        labels = labels.tolist()
        k = torch.tensor(labels[i])
        k = k.view(1,-1)
        k = k.to('cuda')
        loss = criterion(outputs, k)
        #loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_acc += accuracy(outputs, k)

        if i % 80 == 79:
            print('epoch: [%d/%d] train_loss: %.5f train_acc: %.5f' % (
                epoch + 1, epochs, running_loss / 80, running_acc / 80))
            running_loss = 0.0

print("learning finish")

RuntimeError: ignored

# **모델 새로 쌓기**

In [ ]:
# set parameters
learning_rate = 0.0004
num_epoch = 10


# CNN 모델학습
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels =16,kernel_size = 5),   # [4,3,32,32]->[16,3,28,28]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,16,28,28]->[4,16,14,14]
            nn.Conv2d(in_channels = 16,out_channels =32,kernel_size = 5),  # [4,16,14,14]->[4,32,10,10]
            nn.ReLU(),
            nn.MaxPool2d(2,2),                                             # [4,32,10,10]->[4,32,5,5]
            nn.Conv2d(32,64,5,padding=2),                                  # [4,32,5,5]->[4,64,5,5]
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
      out = self.layer(x)
      return out

net = Net()

# 손실함수와 옵티마이저 정하기
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
X_train.shape

torch.Size([1015, 3, 145, 145])

In [ ]:
X_train = X_train.reshape(1015,3,145,145)

In [ ]:
y_train.shape

(1015, 1)

In [ ]:
train = torch.tensor([1015,145,145,3,1015])
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
train[0] = X_train
train[1] = y_train

RuntimeError: ignored

In [ ]:
#신경망 학습하기
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.
    running_loss = 0.0
    for i, data in enumerate(X_train, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs = data
        labels = ['Closed', 'Open']

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: ignored